# Welcome to the AWS GOES data notebook!
#### **Audience:** Anybody with a computer and access to at least 4GB of memory.
#### **Intent:** Build familiarity with GOES-R data for use in scientific research.
#### **Outcome:** A saveable animation of GOES data for a selected region, time range, and product.           

This notebook is an introduction to accessing GOES-R (GOES-16, GOES-17, and GOES-18) satellite data from the AWS Cloud. No coding experience is required to complete this notebook, as all options are presented as Widgets, where you can select your desired product from a drop-down menu. As such, this notebook is meant for workshops, classrooms, or places where you don't want to spend time de-bugging code. 

# Importing

In [ ]:
import ipywidgets as widgets
import requests
import netCDF4
import s3fs
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import cartopy.crs as ccrs
from cartopy import feature as cf
import xarray
from datetime import datetime

# Defining

In [ ]:
def aws_satellite():
    
    global satellite_choice

    # Formatting settings for drop-down menus
    style = {'description_width':'120px'}
    layout = widgets.Layout(width='325px')

    # Display drop-down menus
    print("Select the satellite you'd like to use, then run the next cell to select a date.")
    satellite_choice = widgets.Dropdown(options=[('GOES-16 (East)', "noaa-goes16"), ("GOES-17 (Retired West)",'noaa-goes17'), ("GOES-18 (West)",'noaa-goes18')], description='GOES satellite:', style=style, layout=layout)

    return satellite_choice

In [ ]:
def aws_product(satellite):
    
    global product_choice, scan_choice
        
    aws = s3fs.S3FileSystem(anon=True)
        
    # Formatting settings for drop-down menus
    style = {'description_width':'120px'}
    layout = widgets.Layout(width='325px')

    # Display drop-down menus
    print("Select the satellite product you'd like to use, then run the next cell to select a date.")
    product_choice = widgets.Dropdown(options=[("Aerosol Detection", "ABI-L2-ADP"), ("Cloud Top Height", "ABI-L2-ACHA"), ("Cloud Top Temperature", "ABI-L2-ACHT"),("Land Surface Temperature", "ABI-L2-LST"),("Lightning Detection", "GLM-L2-LCFA"),("True Color Data", "ABI-L2-MCMIP"),("Snow Cover", "ABI-L2-FSC")], description='GOES product:', style=style, layout=layout)
    scan_choice = widgets.Dropdown(options=[("Meso 1","M1"),("Meso 2","M2"),("CONUS/PACUS","C")], description='GOES scan:', style=style, layout=layout)
    
    display(product_choice, scan_choice)

In [ ]:
def aws_date(satellite, product, scan):
    
    global year, day, shour, smin, ehour, emin
            
    aws = s3fs.S3FileSystem(anon=True)
    
    # Search available year directories by product
    if product == "GLM-L2-LCFA": # this product only has one scan type
        years = aws.ls(f'{satellite}/{product}/', refresh=True)  
    else: 
        years = aws.ls(f'{satellite}/{product}{scan[0]}/', refresh=True)  
    
    # Date options to be shown in the drop-down menus 
    year_options = []
    
    for y in years:
        year_options.append(y[-4:])
        
    year_days = range(1,367,1)
    
    # Formatting settings for drop-down menus
    style = {'description_width':'120px'}
    layout = widgets.Layout(width='325px')
    
    # Display drop-down menus
    year = widgets.Dropdown(options=[y for y in year_options], description='Year:', style=style, layout=layout)
    day = widgets.Dropdown(options=[d for d in year_days], description='Day of year:', style=style, layout=layout)
    shour = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23')], description='Start Hour:', style=style, layout=layout)
    smin = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23'), ('24'), ('25'), ('26'), ('27'), ('28'), ('29'), ('30'), ('31'), ('32'), ('33'), ('34'), ('35'), ('36'), ('37'), ('38'), ('39'), ('40'), ('41'), ('42'), ('43'), ('44'), ('45'), ('46'), ('47'), ('48'), ('49'), ('50'), ('51'), ('52'), ('53'), ('54'), ('55'), ('56'), ('57'), ('58'), ('59')], description='Start Minutes:', style=style, layout=layout)
    ehour = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23')], description='End Hour:', style=style, layout=layout)
    emin = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23'), ('24'), ('25'), ('26'), ('27'), ('28'), ('29'), ('30'), ('31'), ('32'), ('33'), ('34'), ('35'), ('36'), ('37'), ('38'), ('39'), ('40'), ('41'), ('42'), ('43'), ('44'), ('45'), ('46'), ('47'), ('48'), ('49'), ('50'), ('51'), ('52'), ('53'), ('54'), ('55'), ('56'), ('57'), ('58'), ('59')], description='End Minutes:', style=style, layout=layout)

    start_time = widgets.HBox([shour, smin])
    end_time = widgets.HBox([ehour, emin])

    print("Select your chosen date and time, chosen from the orbit track, then run the next cell.")
    display(year, day)
    display(start_time, end_time)

In [ ]:
def aws_data(satellite, product, scan, year, day, shour, ehour, smin, emin):

    global fnames
    
    aws = s3fs.S3FileSystem(anon=True)
    fnames = []
    
    # Format date strings
    if len(str(day)) == 1:
        doy = (f"00{day}")
    elif len(str(day)) == 2:
        doy = (f"0{day}")
    elif len(str(day)) == 3:
        doy = (f"{day}")
            
    # Iterate through all desired hours to find files
    for hour in range(int(shour),int(ehour)+1):
        if len(str(hour)) == 1:
            hour = (f"0{hour}")
        if len(str(hour)) == 2:
            hour = (f"{hour}")
        
        if product == "GLM-L2-LCFA": # this product only has one scan type
            data_files = aws.ls(f'{satellite}/{product}/{year}/{doy}/{hour}/', refresh=True)  
        else:
            data_files = aws.ls(f'{satellite}/{product}{scan[0]}/{year}/{doy}/{hour}/', refresh=True)  
            
        # Extracting files for chosen time
        for file in data_files:
            file_start = file.split('_')[-3][8:12]
            file_end = file.split('_')[-2][8:12]
            if ((int(file_start[0:4]) >= int((str(shour) + str(smin)))) and (int(file_end[0:4]) <= int((str(ehour) + str(emin))))):
                fnames.append(file)
    
    # Disclaimer for missing product
    if product == "ABI-L2-ACHT" and scan == "C": # this product only has meso scan types
            print("Cloud top temperature does not have a CONUS/PACUS scan. Please select the meso scan, or try a new product.")
    
    # Finding the correst meso scan
    if scan[0] == "M" and (product != "GLM-L2-LCFA"):
        scan_string = product[7:] + "M" + scan[1]
        fnames = [x for x in fnames if scan_string in x]
        print(f"Retrieved {len(fnames)} files from your criteria.")
    else:
        print(f"Retrieved {len(fnames)} files from your criteria.")

In [ ]:
def aws_plot(satellite, product):        
        #for f in range(0, len(fnames)):  # Uncomment and unindent to plot all images
        f = 0 # Delete this line if you're plotting all images

        print(f"Plotting the first of {len(fnames)} files.")

        fname = (fnames[f])[12:]

        # Load dataset
        resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
        nc = netCDF4.Dataset(fname, memory = resp.content)

        dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
        dataset.load()

        # Aerosol Detection! 
        if product == "ABI-L2-ADP":
            dat = dataset.metpy.parse_cf('Smoke')

            # Identify variable
            smoke = dataset['Smoke']
            dust = dataset['Dust']
            
            # Combine smoke and dust variables into one plot
            plotting_var = (2*smoke) + dust
            x=np.array(plotting_var) 

            # Identify plotting coordinates
            x = dat.x
            y = dat.y
            geos = dat.metpy.cartopy_crs

            # Plotting
            fig = plt.figure(figsize=(12, 7), dpi=100)
            ax = plt.axes(projection=geos)
            cmap = plt.get_cmap('Set3', 4)
            plot = plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, cmap = cmap, interpolation = 'none')
            cbar = fig.colorbar(plot, ticks=[0, 1, 2, 3])
            cbar.ax.set_yticklabels(['No Detectable Aerosol', 'Dust', 'Smoke', 'Smoke and Dust'])
            title = "Aerosol Detection"

        # Cloud Top Height!
        elif product == "ABI-L2-ACHA":
            dat = dataset.metpy.parse_cf('HT')

            # Identify variable 
            plotting_var = dataset['HT'].data

            # Identify plotting coordinates
            x = dat.x
            y = dat.y
            geos = dat.metpy.cartopy_crs

            # Plotting
            fig = plt.figure(figsize=(12, 7), dpi=100)
            ax = plt.axes(projection=geos)
            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, vmin=0, vmax=18000, interpolation = 'none')   
            units = dataset['HT'].units
            plt.colorbar(cmap = "jet", label = units)
            title = "Cloud Top \nHeight"

        # Cloud Top Temperature!
        elif product == "ABI-L2-ACHT":
            dat = dataset.metpy.parse_cf('TEMP')

            # Identify variable and transform to Celsius
            plotting_var = dataset['TEMP'].data - 273.15

            # Identify plotting coordinates
            x = dat.x
            y = dat.y
            geos = dat.metpy.cartopy_crs

            # Plotting
            fig = plt.figure(figsize=(12, 7), dpi=100)
            ax = plt.axes(projection=geos)
            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, interpolation = 'none')   
            units = "Celsius"
            plt.colorbar(cmap = "jet", label = units)
            title = "Cloud Top \nTemperature"

        # Land Surface Temperature!
        elif product == "ABI-L2-LST":
            dat = dataset.metpy.parse_cf('LST')

            # Identify variable and transform to Celsius
            plotting_var = dataset['LST'].data - 273.15

            # Identify plotting coordinates
            x = dat.x
            y = dat.y
            geos = dat.metpy.cartopy_crs

            # Plotting
            fig = plt.figure(figsize=(12, 7), dpi=100)
            ax = plt.axes(projection=geos)
            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, vmin=-40, vmax=40, interpolation = 'none')   
            units = "Celsius"
            plt.colorbar(cmap = "jet", label = units)
            title = "Land Surface \nTemperature"

        # Lightning Detection!
        elif product == "GLM-L2-LCFA":
            dat = dataset.metpy.parse_cf('flash_energy')

            # Identify variable
            flash_energy = dataset['flash_energy'].data 

            # Identify plotting coordinates
            x_flash = dat.flash_lat
            y_flash = dat.flash_lon
            geos = dat.metpy.cartopy_crs  

            # Plotting
            fig = plt.figure(figsize=(12, 7), dpi=100)
            ax = plt.axes(projection=geos)
            lightning = plt.scatter(y_flash, x_flash,c = flash_energy, s = 5, label = "Flash Energy")
            units = dataset['flash_energy'].units 
            plt.colorbar(lightning, label = units)
            title = "Lightning \nDetection"

        # True color data!
        elif product == "ABI-L2-MCMIP": 
            dat = dataset.metpy.parse_cf('CMI_C02')
            
            # Create RGB dataset
            gamma = 2.2
            r = dataset['CMI_C02'].data; r = np.clip(r, 0, 1); r = np.power(r, 1/gamma)
            g = dataset['CMI_C03'].data; g = np.clip(g, 0, 1); g = np.power(g, 1/gamma)
            b = dataset['CMI_C01'].data; b = np.clip(b, 0, 1); b = np.power(b, 1/gamma)

            # Calculate the "True" Green
            g_true = 0.45 * r + 0.1 * g + 0.45 * b
            g_true = np.clip(g_true, 0, 1) 

            # Create a single RGB image for plotting
            plotting_var = np.dstack((r, g_true, b))

            # Identify plotting coordinates
            x = dat.x
            y = dat.y
            geos = dat.metpy.cartopy_crs

            # Plotting
            fig = plt.figure(figsize=(12, 7), dpi=100)
            ax = plt.axes(projection=geos)
            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, interpolation = 'none')  
            title = "True Color \nData"
        
        # Snow Cover!
        elif product == "ABI-L2-FSC":
            dat = dataset.metpy.parse_cf('FSC')

            # Identify variable and transform to Celsius
            plotting_var = dataset['FSC'].data
            plotting_var[plotting_var==128]=np.nan

            # Identify plotting coordinates
            x = dat.x
            y = dat.y
            geos = dat.metpy.cartopy_crs

            # Plotting
            fig = plt.figure(figsize=(12, 7), dpi=100)
            ax = plt.axes(projection=geos)
            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, vmin=0, vmax=100, interpolation = 'none')   
            units = "Snow Cover (%)"
            plt.colorbar(cmap = "jet", label = units)
            title = "Snow Cover"
            
        nc.close()

        # Formatting for all plots
        ax.coastlines(resolution='50m', color='black', linewidth=0.25)
        ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)

        scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
        plt.title(f"GOES-{satellite[-2:]} {title}", loc='left', fontweight='bold')
        plt.title('{}'.format(scan_start.strftime('%d %B %Y at %H:%M UTC')), loc='right')

        plt.show()
        plt.close("all")

In [ ]:
def aws_animate(satellite, product, scan):
    
    # Aerosol Detection! 
    if product == "ABI-L2-ADP":
        
        fig, ax = plt.subplots()
        
        def animate(i):    
            fname = (fnames[i])[12:]
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('Smoke')
            
            # Identify variable and transform to Celsius
            smoke = dataset['Smoke']
            dust = dataset['Dust']

            plotting_var = (2 * smoke) + dust
            x = np.array(smoke) 

            # Identify plotting coordinates
            x = dat.x
            y = dat.y
            
            geos = dat.metpy.cartopy_crs            
            dataset.close()
            
            fig.clear()
            ax = fig.add_subplot(projection=geos)

            cmap = plt.get_cmap('Set3', 4)
            plot = plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, cmap = cmap, interpolation = 'none')
            cbar = fig.colorbar(plot, ticks=[0, 1, 2, 3])
            cbar.ax.set_yticklabels(['None', 'Dust', 'Smoke', 'Smoke & Dust'])
            title = "Aerosol Detection"
            
            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x.min(), x.max(), y.min(), y.max()], geos)
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')
                        
        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames), interval=200)
        ani.save('GOES_ADP_animation.gif', writer='pillow')
        plt.close("all")

        # Cloud Top Height!
    elif product == "ABI-L2-ACHA":
        
        fig, ax = plt.subplots()
        
        def animate(i):    
            fname = (fnames[i])[12:]
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('HT')
                
            # Identify variable and transform to Celsius
            plotting_var = dataset['HT'].data
            
            # Identify plotting coordinates
            x = dat.x
            y = dat.y

            geos = dat.metpy.cartopy_crs
            dataset.close()
            
            fig.clear()
            ax = fig.add_subplot(projection=geos)

            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, vmin=0, vmax=18000, interpolation = 'none')

            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x.min(), x.max(), y.min(), y.max()], geos)
            title = "Cloud Top Height"
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')
            
            units = dataset['HT'].units
            plt.colorbar(cmap = "jet", label = units)
            
        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames), interval=200)
        ani.save('GOES_CTH_animation.gif', writer='pillow')
        plt.close("all")
        
    # Cloud Top Temperature!
    elif product == "ABI-L2-ACHT":
        
        fig, ax = plt.subplots()
        
        def animate(i):    
            fname = (fnames[i])[12:]
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('TEMP')
                
            # Identify variable and transform to Celsius
            plotting_var = dataset['TEMP'].data - 273.15
            # Identify plotting coordinates
            x = dat.x
            y = dat.y

            geos = dat.metpy.cartopy_crs
            dataset.close()
            
            fig.clear()
            ax = fig.add_subplot(projection=geos)

            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, vmin=-80, vmax=20, interpolation = 'none')

            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x.min(), x.max(), y.min(), y.max()], geos)
            title = "Cloud Top Temperature"
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')
            plt.colorbar(label = "Celsius")
            
        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames), interval=200)
        ani.save('GOES_CTT_animation.gif', writer='pillow')
        plt.close("all")

    # Land Surface Temperature!
    elif product == "ABI-L2-LST":
        
        fig, ax = plt.subplots()
        
        def animate(i):    
            fname = (fnames[i])[12:]
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('LST')
                
            # Identify variable and transform to Celsius
            plotting_var = dataset['LST'].data - 273.15
            
            # Identify plotting coordinates
            x = dat.x
            y = dat.y

            geos = dat.metpy.cartopy_crs   
            dataset.close()

            fig.clear()
            ax = fig.add_subplot(projection=geos)
            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, vmin=-40, vmax=40, interpolation = 'none')

            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x.min(), x.max(), y.min(), y.max()], geos)
            title = "Land Surface Temperature"
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')
            plt.colorbar(label = "Celsius")
            
        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames), interval=200)
        ani.save('GOES_LST_animation.gif', writer='pillow')
        plt.close("all")
        
    # Lightning Detection + True Color Data!
    if product == "GLM-L2-LCFA":

        # Reading in true color data
        fnames_truecolor = []

        day_str = str(day.value)
        if len(day_str) == 1:
            doy = (f"00{day_str}")
        elif len(day_str) == 2:
            doy = (f"0{day_str}")
        elif len(day_str) == 3:
            doy = (f"{day_str}")

        aws = s3fs.S3FileSystem(anon=True)

        for hour in range(int(shour.value),int(ehour.value)+1):
            if len(str(hour)) == 1:
                hour = (f"0{hour}")
            if len(str(hour)) == 2:
                hour = (f"{hour}")

            data_files = aws.ls(f'{satellite}/{"ABI-L2-MCMIP"}{scan[0]}/{year.value}/{doy}/{hour}/', refresh=True)  

            for file in data_files:
                file_start = file.split('_')[-3][8:12]
                file_end = file.split('_')[-2][8:12]
                if ((int(file_start[0:4]) >= int((str(shour.value) + str(smin.value)))) and (int(file_end[0:4]) <= int((str(ehour.value) + str(emin.value))))):
                    fnames_truecolor.append(file)
        
        if scan[0] == "M":
            scan_string = "ABI-L2-MCMIPM" + scan[1]
            fnames_truecolor = [x for x in fnames_truecolor if scan_string in x]    
                    
        fig, ax = plt.subplots()
        
        def animate(i):    
            if len(fnames_truecolor) > 1:
                fname_truecolor = (fnames_truecolor[(round((len(fnames_truecolor)/len(fnames))*i))])[12:]    
            else:
                fname_truecolor = (fnames_truecolor[0])[12:]

            resp_truecolor = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname_truecolor}')
            nc_truecolor = netCDF4.Dataset(fname_truecolor, memory = resp_truecolor.content)
            dataset_truecolor = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc_truecolor))
            dataset_truecolor.load()
            dat_truecolor = dataset_truecolor.metpy.parse_cf('CMI_C02')
            gamma = 2.2
            r = dataset_truecolor['CMI_C02'].data; r = np.clip(r, 0, 1); r = np.power(r, 1/gamma)
            g = dataset_truecolor['CMI_C03'].data; g = np.clip(g, 0, 1); g = np.power(g, 1/gamma)
            b = dataset_truecolor['CMI_C01'].data; b = np.clip(b, 0, 1); b = np.power(b, 1/gamma)

            # Calculate the "True" Green
            g_true = 0.45 * r + 0.1 * g + 0.45 * b
            g_true = np.clip(g_true, 0, 1) 
            plotting_var = np.dstack((r, g_true, b))

            # Identify plotting coordinates
            x_truecolor = dat_truecolor.x
            y_truecolor = dat_truecolor.y

            geos_truecolor = dat_truecolor.metpy.cartopy_crs

            fig.clear()
            ax = fig.add_subplot(projection=geos_truecolor)

            #lightning data 
            fname = (fnames[i])[12:]
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('flash_energy')
            x = dat.flash_lat
            y = dat.flash_lon
            geos = dat.metpy.cartopy_crs 
            
            dataset.close()
            dataset_truecolor.close()

            plt.imshow(plotting_var, origin='upper', extent=(x_truecolor.min(), x_truecolor.max(), y_truecolor.min(), y_truecolor.max()), transform=geos_truecolor)
            ax.scatter(y, x, c = "blue", s=5, transform=geos)

            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x_truecolor.min(), x_truecolor.max(), y_truecolor.min(), y_truecolor.max()], geos_truecolor)
            title = "Lightning Detection"
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')
        
        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames)-1, interval=200)
        ani.save('GOES_lightning.gif', writer='pillow')
        plt.close("all")
        
    # True Color Data!
    elif product == "ABI-L2-MCMIP":
        
        fig, ax = plt.subplots()
        
        def animate(i):    
            fname = (fnames[i])[12:]
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('CMI_C02')
            gamma = 2.2
            r = dataset['CMI_C02'].data; r = np.clip(r, 0, 1); r = np.power(r, 1/gamma)
            g = dataset['CMI_C03'].data; g = np.clip(g, 0, 1); g = np.power(g, 1/gamma)
            b = dataset['CMI_C01'].data; b = np.clip(b, 0, 1); b = np.power(b, 1/gamma)

            # Calculate the "True" Green
            g_true = 0.45 * r + 0.1 * g + 0.45 * b
            g_true = np.clip(g_true, 0, 1) 

            # Create a single RGB image for plotting
            plotting_var = np.dstack((r, g_true, b))

            # Identify plotting coordinates
            x = dat.x
            y = dat.y
            geos = dat.metpy.cartopy_crs
            dataset.close()
            
            fig.clear()
            
            # Plotting
            ax = fig.add_subplot(projection=geos)
            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)  
            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x.min(), x.max(), y.min(), y.max()], geos)
            title = "True Color Data"
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')
            
        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames), interval=200)
        ani.save('GOES_TCD_animation.gif', writer='pillow')
        plt.close("all")
        
    # Snow cover!
    elif product == "ABI-L2-FSC":
        
        fig, ax = plt.subplots()
        
        def animate(i):    
            fname = (fnames[i])[12:]
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('FSC')
                
            # Identify variable and transform to Celsius
            plotting_var = dataset['FSC'].data
            plotting_var[plotting_var==128]=np.nan
            
            # Identify plotting coordinates
            x = dat.x
            y = dat.y

            geos = dat.metpy.cartopy_crs
            dataset.close()
            
            fig.clear()
            
            # Plotting
            ax = fig.add_subplot(projection=geos)
            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, vmin=0, vmax=100, interpolation = 'none')   
            units = "Snow Cover (%)"
            plt.colorbar(cmap = "jet", label = units)

            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x.min(), x.max(), y.min(), y.max()], geos)
            title = "Snow Cover"
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')
            
        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames)-1, interval=200)
        ani.save('GOES_snow.gif', writer='pillow')
        plt.close("all")

# Choosing your variables!

In [ ]:
aws_satellite()

In [ ]:
aws_product(satellite_choice.value)

In [ ]:
aws_date(satellite_choice.value, product_choice.value, scan_choice.value)

In [ ]:
aws_data(satellite_choice.value, product_choice.value, scan_choice.value, year.value, day.value, shour.value, ehour.value, smin.value, emin.value)

In [ ]:
aws_animate(satellite_choice.value, product_choice.value, scan_choice.value)

In [ ]:
aws_plot(satellite_choice.value, product_choice.value)

In [ ]:
# Your analysis here!

**GOES Product Documentation:**
- https://www.goes-r.gov/products/overview.html (Broad explanation of all products)
- https://www.star.nesdis.noaa.gov/goesr/ (Algorithm & product description)
- http://cimss.ssec.wisc.edu/goes/OCLOFactSheetPDFs/ (Quick guides)
- https://github.com/awslabs/open-data-docs/blob/main/docs/noaa/noaa-goes16/README.md (Variables & product names) 
- https://www.goes-r.gov/products/docs/PUG-L2+-vol5.pdf (Technical explanation of each product and variable)

**Coding Sources:**
- https://unidata.github.io/python-gallery/examples/mapping_GOES16_TrueColor.html#sphx-glr-download-examples-mapping-goes16truecolor-py   
- https://unidata.github.io/python-training/gallery/mapping_goes16_truecolor/   
- https://fire.trainhub.eumetsat.int/docs/figure3_GOES17_L1B_and_L2.html   
- https://unidata.github.io/python-training/gallery/mapping_goes16_truecolor/

**CSP Access:**    
AWS: https://registry.opendata.aws/noaa-goes/   
Google: https://console.cloud.google.com/marketplace/product/noaa-public/goes   
Azure: https://microsoft.github.io/AIforEarthDataSets/data/goes-r.html   

The unique component of this Jupyter notebook is that you are not requried to download any datasets -- all data will be pulled directly from the cloud. You can learn more about NOAA's efforts to move more data to the cloud at this site: https://www.noaa.gov/nodd/datasets. As we continute to make more data widely accessible on the cloud, we'll also create more Jupyter notebooks like this one, so anyone can visualize weather and climate data without any cost or restriction. 